In [2]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

In [3]:
mnist_dataset,mnist_info=tfds.load(name="mnist",with_info=True,as_supervised=True)
mnist_dataset

{'train': <PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'test': <PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [4]:
mnist_train,mnist_test=mnist_dataset["train"],mnist_dataset["test"]
num_validation_samples=0.1*mnist_info.splits['train'].num_examples
num_validation_samples=tf.cast(num_validation_samples,tf.int64)
num_validation_samples

<tf.Tensor: shape=(), dtype=int64, numpy=6000>

In [5]:
num_test_samples=mnist_info.splits['test'].num_examples
num_test_samples

10000

In [6]:
num_test_samples=tf.cast(num_test_samples,tf.int64)
num_test_samples

<tf.Tensor: shape=(), dtype=int64, numpy=10000>

In [9]:
#standardization
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image /=255.
    return image,label

In [10]:
scaled_train_and_validation_data=mnist_train.map(scale)
test_data=mnist_test.map(scale)

In [11]:
#shuffling_of data
Buffer_size=10000
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(Buffer_size)

In [14]:
validation_data=shuffled_train_and_validation_data.take(num_validation_samples)
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)

In [15]:
# Batching
Batch_size=100
train_data=train_data.batch(Batch_size)
validation_data=validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)

In [16]:
validation_inputs,validation_targets=next(iter(validation_data))

In [17]:
##outline the model

In [19]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [46]:
input_size=784
output_size=10
hidden_layer_size=200
model=Sequential([
                 Flatten(input_shape=(28,28,1)),
                 Dense(units=hidden_layer_size,activation="relu"),
                 Dense(units=hidden_layer_size,activation="relu"),
#                  Dense(units=hidden_layer_size,activation="relu"),
                 Dense(units=output_size,activation="softmax")
                 ])
model.compile(optimizer="Adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [49]:
num_epochs=10
model.fit(train_data,epochs=num_epochs,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/10
540/540 - 3s - loss: 0.0313 - accuracy: 0.9900 - val_loss: 0.0433 - val_accuracy: 0.9863 - 3s/epoch - 5ms/step
Epoch 2/10
540/540 - 3s - loss: 0.0257 - accuracy: 0.9916 - val_loss: 0.0375 - val_accuracy: 0.9893 - 3s/epoch - 5ms/step
Epoch 3/10
540/540 - 2s - loss: 0.0225 - accuracy: 0.9927 - val_loss: 0.0349 - val_accuracy: 0.9887 - 2s/epoch - 4ms/step
Epoch 4/10
540/540 - 2s - loss: 0.0167 - accuracy: 0.9949 - val_loss: 0.0252 - val_accuracy: 0.9935 - 2s/epoch - 5ms/step
Epoch 5/10
540/540 - 3s - loss: 0.0158 - accuracy: 0.9948 - val_loss: 0.0230 - val_accuracy: 0.9918 - 3s/epoch - 5ms/step
Epoch 6/10
540/540 - 3s - loss: 0.0149 - accuracy: 0.9953 - val_loss: 0.0190 - val_accuracy: 0.9937 - 3s/epoch - 5ms/step
Epoch 7/10
540/540 - 2s - loss: 0.0132 - accuracy: 0.9956 - val_loss: 0.0158 - val_accuracy: 0.9955 - 2s/epoch - 4ms/step
Epoch 8/10
540/540 - 2s - loss: 0.0127 - accuracy: 0.9957 - val_loss: 0.0282 - val_accuracy: 0.9918 - 2s/epoch - 4ms/step
Epoch 9/10
540/540 - 2s 

In [50]:
#testing the model
test_loss,test_accuracy=model.evaluate(test_data)
print(test_loss)
print(test_accuracy)

1/1 [==============================] - 0s 157ms/step - loss: 0.0758 - accuracy: 0.9809
0.07582874596118927
0.98089998960495
